In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sb



from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression


In [2]:
df = pd.read_csv('transaction.csv',sep=";")
df.head(10)

,txn_id,area_type,contract_date,district,floor_area,floor_range,no_of_units,price,prop_type,sale_type,tenure,project_project_id
0,1,Land,220,5,524.3,-,1,5500000.0,Semi-detached,3,Freehold,1
1,2,Land,918,5,308.0,-,1,5000000.0,Semi-detached,3,Freehold,1
2,3,Land,618,5,314.0,-,1,4750000.0,Semi-detached,3,Freehold,1
3,4,Land,921,5,308.0,-,1,5200000.0,Semi-detached,3,Freehold,1
4,5,Strata,321,5,186.0,-,1,1880000.0,Strata Terrace,3,Freehold,2
5,6,Land,320,5,159.3,-,1,2630000.0,Terrace,3,Freehold,2
6,7,Strata,521,4,194.0,01-05,1,2793790.0,Condominium,3,99 yrs lease commencing from 2007,3
7,8,Strata,721,4,194.0,01-05,1,3240240.0,Condominium,3,99 yrs lease commencing from 2007,3
8,9,Strata,721,4,195.0,01-05,1,3125000.0,Condominium,3,99 yrs lease commencing from 2007,3
9,10,Strata,521,4,194.0,01-05,1,3178150.0,Condominium,3,99 yrs lease commencing from 2007,3


In [3]:
def contract_month(x):
    x=str(x)
    return int(x[:-2])

def contract_year(x):
    x=str(x)
    return int(x[-2:])

def floor_range(x):
    if (x=='-'):
        return 1
    else:
        y=x.split("-")[0]
        if (y.isdigit() == True):
            return int(y)
        else:
            return 0


def floor_area_per_unit():
    df['floor_area_per_unit'] = df['floor_area'] / df['no_of_units']

def price_per_unit():
    df['price_per_unit'] = df['price'] / df['no_of_units']
    df['unit_price_per_unit'] = df['price'] / df['floor_area']

def top(x):
    x=str(x)
    if(x == 'Freehold'):
        return 0
    else:
        return int(x[-4:])

def tenure_(x):
    x=str(x)
    if(x == 'Freehold'):
        return 999999
    else:
        return int(x.split(" ")[0])


df['contract_month'] = df['contract_date'].apply(contract_month)
df['contract_year'] = df['contract_date'].apply(contract_year)
df['floor_range'] = df['floor_range'].apply(floor_range)
floor_area_per_unit()
price_per_unit()
df.dropna(axis=0, inplace=True)
# df['tenure'].fillna(method='ffill',axis=0,inplace = True)
df['top'] = df['tenure'].apply(top)
df['tenure_'] = df['tenure'].apply(tenure_)


In [4]:
# df.head(50)

In [5]:
df[df.isna().T.any()]

,txn_id,area_type,contract_date,district,floor_area,floor_range,no_of_units,price,prop_type,sale_type,tenure,project_project_id,contract_month,contract_year,floor_area_per_unit,price_per_unit,unit_price_per_unit,top,tenure_


In [6]:
df1=df[['project_project_id','top',	'tenure_']]
df1.groupby('project_project_id').nunique().sort_values(['top'], ascending = False)

,top,tenure_
project_project_id,,
2408,5,3
169,4,3
2273,4,3
3285,4,2
4024,3,1
...,...,...
1383,1,1
1384,1,1
1385,1,1


In [7]:
x=df[['project_project_id','district','floor_area_per_unit','floor_range','top','tenure_','contract_year','contract_month']]
# x=df[['project_project_id','district','floor_area_per_unit','floor_range','top','tenure_']]
y=df[['price_per_unit']]


In [8]:
linReg = LinearRegression()
linReg.fit(x, y)

LinearRegression()

In [9]:
print(linReg.intercept_)
print(linReg.coef_)
print(linReg.score(x, y))

[-3109478.13338569]
[[-4.42632956e+01 -3.24768684e+04  1.45344777e+04  1.95830169e+04
   9.74118866e+02  2.33792929e+00  7.84672598e+04  7.16671171e+03]]
0.7477104944401498


In [10]:
x_pred = np.array([[3,4,194,1,2007,99,22,5]])
# x_pred = np.array([[1,5,524.3,1,0,0,20,2]])
# x_pred = np.array([[225,4,370,21,2009,99,20,7]])


In [11]:
y_predict=linReg.predict(x_pred)
y_predict

array([[3317154.57834436]])

In [14]:
from joblib import dump, load
dump(linReg, 'model.joblib')

['model.joblib']

In [15]:
model_in = load('model.joblib')
model_in.predict(np.array([[3,4,194,1,2007,99,22,5]]))

array([[3317154.57834436]])